In [1]:
# ==== Train 4 LightGBM heads on Burst 1 (3 train days / 1 valid day / 5 left for inference later) ====
from pathlib import Path
import json, os, math, numpy as np, pandas as pd, lightgbm as lgb
from sklearn.metrics import roc_auc_score, average_precision_score, log_loss
from sklearn.linear_model import LogisticRegression

In [2]:
# ---------- Paths ----------
BASE   = Path("/Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data")
PREP   = BASE / "prepared" / 。
OUTDIR = BASE / "models" / "burst1"
OUTDIR.mkdir(parents=True, exist_ok=True)

# ---------- Load prepared features ----------
df = pd.read_parquet(PREP)
assert "burst_id" in df.columns, "burst_id missing. Run the data-prep notebook first."
assert "ts" in df.columns, "ts missing (datetime)."


In [3]:
# ---------- Identify Burst 1 9 days and split 3/1/5 ----------
b1 = df[df["burst_id"] == 1].copy()
b1["day"] = b1["ts"].dt.normalize()
days = np.sort(b1["day"].unique())
if len(days) < 9:
    raise ValueError(f"Burst 1 has {len(days)} days; need at least 9 for 3/1/5 split.")
train_days = days[:3]
valid_days = days[3:4]
hold_days  = days[4:9]    # we won't use these here (reserved for inference step later)

print("Burst 1 days:", pd.Series(days).dt.strftime("%Y-%m-%d").tolist())
print("Train days  :", pd.Series(train_days).dt.strftime("%Y-%m-%d").tolist())
print("Valid day   :", pd.Series(valid_days).dt.strftime("%Y-%m-%d").tolist())
print("Hold (5d)   :", pd.Series(hold_days).dt.strftime("%Y-%m-%d").tolist())

Burst 1 days: ['2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08', '2020-07-09', '2020-07-10', '2020-07-11', '2020-07-12']
Train days  : ['2020-07-04', '2020-07-05', '2020-07-06']
Valid day   : ['2020-07-07']
Hold (5d)   : ['2020-07-08', '2020-07-09', '2020-07-10', '2020-07-11', '2020-07-12']


In [4]:
# ---------- Build feature list (NO LEAKAGE) ----------
label_cols = ["y_complete","y_long","y_rewatch","y_neg"]

# IDs / helpers we never train on
key_cols   = ["user_id","video_id","timestamp","ts","day","session"]

# columns that leak the target or use future info
leak_cols = {
    "play_duration",      # outcome
    "watch_ratio",        # outcome
    "sess_rank",          # needs full session
    "sess_len",           # needs full session
    "sess_rank_frac",     # needs full session
}

drop_cols = set(label_cols + key_cols) | leak_cols
feat_cols = [c for c in b1.columns if c not in drop_cols]

# mark categoricals (keep pandas 'category' dtype)
cat_cols = [c for c in feat_cols
            if getattr(b1[c].dtype, "name", "") == "category" or str(b1[c].dtype) == "category"]

print(f"{len(feat_cols)} feature cols  |  removed leaks: {sorted(leak_cols & set(b1.columns))}")


64 feature cols  |  removed leaks: ['play_duration', 'sess_len', 'sess_rank', 'sess_rank_frac', 'watch_ratio']


In [5]:
# ---------- Make splits ----------
train = b1[b1["day"].isin(train_days)]
valid = b1[b1["day"].isin(valid_days)]

# Small sanity
for y in label_cols:
    print(f"{y}: train pos={int(train[y].sum())}/{len(train)}, valid pos={int(valid[y].sum())}/{len(valid)}")

y_complete: train pos=349572/489948, valid pos=190256/270269
y_long: train pos=239061/489948, valid pos=127574/270269
y_rewatch: train pos=44867/489948, valid pos=23392/270269
y_neg: train pos=62558/489948, valid pos=35086/270269


In [6]:
feat_cols

['u_user_active_degree',
 'u_is_lowactive_period',
 'u_is_live_streamer',
 'u_is_video_author',
 'u_follow_user_num',
 'u_follow_user_num_range',
 'u_fans_user_num',
 'u_fans_user_num_range',
 'u_friend_user_num',
 'u_friend_user_num_range',
 'u_register_days',
 'u_register_days_range',
 'u_onehot_feat0',
 'u_onehot_feat1',
 'u_onehot_feat2',
 'u_onehot_feat3',
 'u_onehot_feat4',
 'u_onehot_feat5',
 'u_onehot_feat6',
 'u_onehot_feat7',
 'u_onehot_feat8',
 'u_onehot_feat9',
 'u_onehot_feat10',
 'u_onehot_feat11',
 'u_onehot_feat12',
 'u_onehot_feat13',
 'u_onehot_feat14',
 'u_onehot_feat15',
 'u_onehot_feat16',
 'u_onehot_feat17',
 'u_follow_user_num_log1p',
 'u_fans_user_num_log1p',
 'u_friend_user_num_log1p',
 'u_register_days_log1p',
 'i_author_id',
 'i_video_type',
 'i_upload_type',
 'i_visible_status',
 'video_width',
 'video_height',
 'i_music_id',
 'i_video_tag_id',
 'i_video_duration_s',
 'i_aspect_ratio',
 'i_age_since_upload_days',
 'i_top_category_id',
 'ctx_hour_sin',
 'ctx_

In [18]:
len(feat_cols)

64

In [7]:
# ---------- Training helper (per head) ----------
def train_head(label: str):
    X_tr, y_tr = train[feat_cols], train[label].astype("int8")
    X_va, y_va = valid[feat_cols], valid[label].astype("int8")

    # scale_pos_weight from train only
    pos = int(y_tr.sum()); neg = int(len(y_tr) - pos)
    spw = (neg / max(pos, 1)) if pos > 0 else 1.0

    dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols, free_raw_data=False)
    dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols, reference=dtrain, free_raw_data=False)

    params = dict(
        objective="binary",
        metric="auc",
        learning_rate=0.05,
        num_leaves=127,
        max_depth=7,
        min_data_in_leaf=200,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=1,
        lambda_l2=2.0,
        max_bin=255,
        n_jobs=-1,
        verbose=-1,
        scale_pos_weight=spw,
        seed=2025,
    )

    # ---> use callbacks for early stopping + logging (compatible with older LightGBM)
    callbacks = [
        lgb.early_stopping(stopping_rounds=100, verbose=True),
        lgb.log_evaluation(period=200),
    ]

    booster = lgb.train(
        params,
        dtrain,
        num_boost_round=4000,
        valid_sets=[dtrain, dvalid],
        valid_names=["train","valid"],
        callbacks=callbacks,
    )

    # predictions
    pred_tr_raw = booster.predict(X_tr, num_iteration=booster.best_iteration)
    pred_va_raw = booster.predict(X_va, num_iteration=booster.best_iteration)
    margin_va   = booster.predict(X_va, num_iteration=booster.best_iteration, raw_score=True)

    def safe_metrics(y, p):
        if y.nunique() < 2:
            return dict(auc=float("nan"), ap=float("nan"), logloss=float("nan"))
        return dict(
            auc=roc_auc_score(y, p),
            ap=average_precision_score(y, p),
            logloss=log_loss(y, np.clip(p, 1e-6, 1-1e-6)),
        )

    m_tr_raw = safe_metrics(y_tr, pred_tr_raw)
    m_va_raw = safe_metrics(y_va, pred_va_raw)

    # Platt calibration on valid
    calibrator = {"type": "platt", "coef": 1.0, "intercept": 0.0}
    if y_va.nunique() == 2 and (np.std(margin_va) > 1e-8):
        lr = LogisticRegression(solver="lbfgs", C=1e6, max_iter=1000, fit_intercept=True)
        lr.fit(margin_va.reshape(-1,1), y_va.to_numpy())
        a = float(lr.coef_.ravel()[0]); b = float(lr.intercept_.ravel()[0])
        calibrator.update({"coef": a, "intercept": b})
        pred_va_cal = 1.0 / (1.0 + np.exp(-(a*margin_va + b)))
        m_va_cal = safe_metrics(y_va, pred_va_cal)
    else:
        m_va_cal = m_va_raw

    # save artifacts
    head_dir = OUTDIR / label
    head_dir.mkdir(parents=True, exist_ok=True)

    model_path = head_dir / f"lgb_b1_{label}.txt"
    booster.save_model(str(model_path), num_iteration=booster.best_iteration)

    imp = pd.DataFrame({
        "feature": booster.feature_name(),
        "gain": booster.feature_importance(importance_type="gain"),
        "split": booster.feature_importance(importance_type="split"),
    }).sort_values("gain", ascending=False)
    imp_path = head_dir / f"feature_importance.csv"
    imp.to_csv(imp_path, index=False)

    meta = {
        "burst": 1,
        "label": label,
        "best_iteration": int(booster.best_iteration),
        "class_balance": {"pos": pos, "neg": neg, "scale_pos_weight": spw},
        "metrics": {"train_raw": m_tr_raw, "valid_raw": m_va_raw, "valid_cal": m_va_cal},
        "params": params,
        "n_features": len(feat_cols),
        "n_categorical": len(cat_cols),
        "train_days": pd.Series(train_days).dt.strftime("%Y-%m-%d").tolist(),
        "valid_days": pd.Series(valid_days).dt.strftime("%Y-%m-%d").tolist(),
        "hold_days":  pd.Series(hold_days).dt.strftime("%Y-%m-%d").tolist(),
        "model_path": str(model_path),
        "importance_path": str(imp_path),
    }
    (head_dir / "meta.json").write_text(json.dumps(meta, indent=2))
    (head_dir / "calibrator.json").write_text(json.dumps(calibrator, indent=2))
    (head_dir / "features.json").write_text(json.dumps({"feat_cols": feat_cols, "cat_cols": cat_cols}, indent=2))

    print(f"\n[{label}] best_iter={meta['best_iteration']}  "
          f"Train AUC={m_tr_raw['auc']:.4f} | "
          f"Valid AUC raw={m_va_raw['auc']:.4f} cal={m_va_cal['auc']:.4f}")
    print(f"[{label}] Valid AP raw={m_va_raw['ap']:.4f} cal={m_va_cal['ap']:.4f}   "
          f"LogLoss raw={m_va_raw['logloss']:.4f} cal={m_va_cal['logloss']:.4f}")
    print(f"Saved → {model_path}\n")

    return booster, meta, imp


In [8]:
# ---------- Train all four heads (no inference yet) ----------
models = {}
for label in ["y_complete","y_long","y_rewatch","y_neg"]:
    print("\n" + "="*90)
    print(f"Training: {label}")
    booster, meta, imp = train_head(label)
    models[label] = {"model": booster, "meta": meta, "importance": imp}

print("\nDone (Burst 1 training + calibration). Next step would be inference on days D5–D9.")



Training: y_complete
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.876591	valid's auc: 0.846778
Early stopping, best iteration is:
[195]	train's auc: 0.876031	valid's auc: 0.84681

[y_complete] best_iter=195  Train AUC=0.8760 | Valid AUC raw=0.8468 cal=0.8468
[y_complete] Valid AP raw=0.9259 cal=0.9259   LogLoss raw=0.4702 cal=0.4263
Saved → /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/models/burst1/y_complete/lgb_b1_y_complete.txt


Training: y_long
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.848966	valid's auc: 0.819844
[400]	train's auc: 0.86547	valid's auc: 0.820828
Early stopping, best iteration is:
[469]	train's auc: 0.869725	valid's auc: 0.820953

[y_long] best_iter=469  Train AUC=0.8697 | Valid AUC raw=0.8210 cal=0.8210
[y_long] Valid AP raw=0.8075 cal=0.8075   LogLoss raw=0.5147 cal=0.5143
Saved → /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/models/burst1/y_long/lgb_b1_y_lo

In [9]:
# ==== Test (Holdout) evaluation on Burst-1 Days D5–D9 ====
import json, numpy as np, pandas as pd, lightgbm as lgb
from pathlib import Path
from sklearn.metrics import roc_auc_score, average_precision_score, log_loss, brier_score_loss

# 0) Build test slice
test = b1[b1["day"].isin(hold_days)].copy()
X_te = test[feat_cols]

def load_and_score(head: str):
    """Load the trained head model + Platt calibrator, and return raw & calibrated probs on the test set."""
    head_dir = OUTDIR / head
    booster = lgb.Booster(model_file=str(head_dir / f"lgb_b1_{head}.txt"))
    with open(head_dir / "calibrator.json") as f:
        calib = json.load(f)

    # raw prob and raw margin on test
    p_raw = booster.predict(X_te, num_iteration=booster.best_iteration)
    margin = booster.predict(X_te, num_iteration=booster.best_iteration, raw_score=True)

    # Platt calibration
    a = float(calib.get("coef", 1.0))
    b = float(calib.get("intercept", 0.0))
    p_cal = 1.0 / (1.0 + np.exp(-(a * margin + b)))
    return booster, p_raw, p_cal

def metrics(y, p):
    y = y.astype("int8").to_numpy()
    p = np.clip(p, 1e-6, 1-1e-6)
    out = {}
    if y.min() != y.max():
        out["auc"] = roc_auc_score(y, p)
        out["ap"]  = average_precision_score(y, p)
    else:
        out["auc"] = np.nan
        out["ap"]  = np.nan
    out["logloss"] = log_loss(y, p)
    out["brier"]   = brier_score_loss(y, p)
    return out

# 1) Overall metrics on test (D5–D9)
report = {}
for head in ["y_complete","y_long","y_rewatch","y_neg"]:
    _, p_raw, p_cal = load_and_score(head)
    y = test[head]
    report[head] = {"raw": metrics(y, p_raw), "cal": metrics(y, p_cal)}

# 2) Per-day metrics (use calibrated probs)
per_day_rows = []
for head in ["y_complete","y_long","y_rewatch","y_neg"]:
    _, _, p_cal = load_and_score(head)
    tmp = test[["day", head]].copy()
    tmp["p_cal"] = p_cal
    for d, g in tmp.groupby("day"):
        m = metrics(g[head], g["p_cal"])
        per_day_rows.append([head, d.strftime("%Y-%m-%d"), m["auc"], m["ap"], m["logloss"], m["brier"]])

# 3) Pretty print
pd.set_option("display.float_format", lambda x: f"{x:0.4f}")
rows = []
for head in ["y_complete","y_long","y_rewatch","y_neg"]:
    rows.append([
        head,
        report[head]["raw"]["auc"],   report[head]["cal"]["auc"],
        report[head]["raw"]["ap"],    report[head]["cal"]["ap"],
        report[head]["raw"]["logloss"], report[head]["cal"]["logloss"],
        report[head]["raw"]["brier"],   report[head]["cal"]["brier"],
    ])
test_table = pd.DataFrame(rows, columns=[
    "head","AUC_raw","AUC_cal","AP_raw","AP_cal","LogLoss_raw","LogLoss_cal","Brier_raw","Brier_cal"
])

print("=== Burst-1 Holdout (Days D5–D9) — Overall ===")
display(test_table)

per_day = pd.DataFrame(per_day_rows, columns=["head","day","AUC_cal","AP_cal","LogLoss_cal","Brier_cal"])
print("\n=== Per-day metrics (calibrated) ===")
display(per_day.sort_values(["head","day"]))

# 4) Save artifacts
metrics_dir = OUTDIR / "evaluation"
metrics_dir.mkdir(parents=True, exist_ok=True)
test_table.to_csv(metrics_dir / "test_overall.csv", index=False)
per_day.to_csv(metrics_dir / "test_per_day.csv", index=False)

with open(metrics_dir / "test_overall.json", "w") as f:
    json.dump(report, f, indent=2)

print(f"\nSaved overall and per-day metrics to: {metrics_dir}")


=== Burst-1 Holdout (Days D5–D9) — Overall ===


,head,AUC_raw,AUC_cal,AP_raw,AP_cal,LogLoss_raw,LogLoss_cal,Brier_raw,Brier_cal
0,y_complete,0.8447,0.8447,0.9266,0.9266,0.4681,0.4248,0.1545,0.1379
1,y_long,0.8086,0.8086,0.7949,0.7949,0.5294,0.5292,0.1782,0.1781
2,y_rewatch,0.8123,0.8123,0.3388,0.3388,0.4246,0.2393,0.1343,0.0678
3,y_neg,0.8095,0.8095,0.4115,0.4115,0.4474,0.3040,0.1493,0.0914



=== Per-day metrics (calibrated) ===


,head,day,AUC_cal,AP_cal,LogLoss_cal,Brier_cal
0,y_complete,2020-07-08,0.8557,0.9308,0.4154,0.1346
1,y_complete,2020-07-09,0.8473,0.9272,0.4237,0.1378
2,y_complete,2020-07-10,0.8488,0.9252,0.4239,0.1372
3,y_complete,2020-07-11,0.8411,0.9291,0.4216,0.1365
4,y_complete,2020-07-12,0.8243,0.9176,0.4443,0.1451
5,y_long,2020-07-08,0.8171,0.8050,0.5200,0.1742
6,y_long,2020-07-09,0.8118,0.7927,0.5243,0.1763
7,y_long,2020-07-10,0.8110,0.7945,0.5261,0.1770
8,y_long,2020-07-11,0.8046,0.7980,0.5344,0.1802
9,y_long,2020-07-12,0.7940,0.7798,0.5450,0.1846



Saved overall and per-day metrics to: /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/models/burst1/evaluation


In [10]:
b1.head()

,user_id,video_id,play_duration,timestamp,watch_ratio,ts,session,sess_rank,sess_len,sess_rank_frac,...,hist_last3_wr_var,hist_ema_complete,hist_ema_wr_mean,hist_prev_within_sess_wr_slope,hist_author_recency_days,hist_author_recency_log1p,hist_user_cat_complete_ema,hist_cat_entropy,burst_id,day
0,0,3649,13838,1593878903.4380,1.2734,2020-07-04 16:08:23.437999964,1,1,6,0.1667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-07-04
1,0,9598,13665,1593879221.2970,1.2441,2020-07-04 16:13:41.296999931,1,2,6,0.3333,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,-0.0000,1,2020-07-04
2,0,5262,851,1593879366.6870,0.1076,2020-07-04 16:16:06.687000036,1,3,6,0.5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0000,1,2020-07-04
3,0,1963,862,1593879626.7920,0.0899,2020-07-04 16:20:26.792000055,1,4,6,0.6667,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0000,-0.0000,1,2020-07-04
4,0,8234,858,1593880985.1280,0.0780,2020-07-04 16:43:05.128000020,1,5,6,0.8333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0000,1,2020-07-04


In [11]:
# total number of (user, session) pairs
total_sessions = b1.drop_duplicates(["user_id", "session"]).shape[0]
print("Total sessions:", total_sessions)

# sessions per day (using calendar day from ts)
sess_per_day = (
    b1.assign(day=b1["ts"].dt.normalize())
      .drop_duplicates(["user_id", "session", "day"])
      .groupby("day").size().rename("n_sessions").reset_index()
)
print(sess_per_day)

# sessions with at least 10 exposures
sess_sizes = b1.groupby(["user_id","session"]).size().rename("n_exposures")
n10 = (sess_sizes >= 10).sum()
print("Sessions with ≥10 exposures:", n10)


# Unique users in b1
n_users = b1["user_id"].nunique()

# Unique videos that appeared (shown) in b1
n_videos = b1["video_id"].nunique()

print("Unique users:", n_users)
print("Unique videos shown:", n_videos)


n_days = b1["day"].nunique()
print("Unique days:", n_days)

Total sessions: 152456
         day  n_sessions
0 2020-07-04        6421
1 2020-07-05       20834
2 2020-07-06       20988
3 2020-07-07       20707
4 2020-07-08       20970
5 2020-07-09       20722
6 2020-07-10       20541
7 2020-07-11       20831
8 2020-07-12       14677
Sessions with ≥10 exposures: 62152
Unique users: 7041
Unique videos shown: 2919
Unique days: 9


In [12]:
print(len(b1.columns), "columns")
list(b1.columns)

79 columns


['user_id',
 'video_id',
 'play_duration',
 'timestamp',
 'watch_ratio',
 'ts',
 'session',
 'sess_rank',
 'sess_len',
 'sess_rank_frac',
 'y_complete',
 'y_long',
 'y_rewatch',
 'y_neg',
 'u_user_active_degree',
 'u_is_lowactive_period',
 'u_is_live_streamer',
 'u_is_video_author',
 'u_follow_user_num',
 'u_follow_user_num_range',
 'u_fans_user_num',
 'u_fans_user_num_range',
 'u_friend_user_num',
 'u_friend_user_num_range',
 'u_register_days',
 'u_register_days_range',
 'u_onehot_feat0',
 'u_onehot_feat1',
 'u_onehot_feat2',
 'u_onehot_feat3',
 'u_onehot_feat4',
 'u_onehot_feat5',
 'u_onehot_feat6',
 'u_onehot_feat7',
 'u_onehot_feat8',
 'u_onehot_feat9',
 'u_onehot_feat10',
 'u_onehot_feat11',
 'u_onehot_feat12',
 'u_onehot_feat13',
 'u_onehot_feat14',
 'u_onehot_feat15',
 'u_onehot_feat16',
 'u_onehot_feat17',
 'u_follow_user_num_log1p',
 'u_fans_user_num_log1p',
 'u_friend_user_num_log1p',
 'u_register_days_log1p',
 'i_author_id',
 'i_video_type',
 'i_upload_type',
 'i_visible_sta

In [13]:
# build empirical distribution for each session

import numpy as np
import pandas as pd
from pathlib import Path

# Config
cat_col = "i_top_category_id"     # use the ID column for categories
eta = 1e-2                        # smoothing
min_exposures = 10                 # flag "trustworthy" sessions
OUT_PREP = Path(OUTDIR).parents[1] / "prepared"
OUT_PREP.mkdir(parents=True, exist_ok=True)

# 1) Treat each row in b1 as an exposure; keep (user, session, category)
expo = b1.loc[b1[cat_col].notna(), ["user_id","session",cat_col,"ts"]].copy()
expo.sort_values(["user_id","session","ts"], inplace=True, kind="mergesort")

# 2) Counts per (user, session, category) and totals per (user, session)
g = (expo.groupby(["user_id","session",cat_col], as_index=False)
          .size().rename(columns={"size":"n_exposed"}))
tot = (g.groupby(["user_id","session"], as_index=False)["n_exposed"]
         .sum().rename(columns={"n_exposed":"n_total"}))

emp = g.merge(tot, on=["user_id","session"], how="left")

# 3) #categories observed in the session (for smoothed denominator)
K = (emp.groupby(["user_id","session"], observed=True)[cat_col]
        .nunique()
        .rename("K_session")
        .reset_index())
emp = emp.merge(K, on=["user_id","session"], how="left")

# 4) Raw and smoothed probabilities
emp["p_empirical_cat"] = (emp["n_exposed"] / emp["n_total"]).astype("float32")
emp["p_empirical_cat_sm"] = (
    (emp["n_exposed"] + eta) / (emp["n_total"] + eta * emp["K_session"])
).astype("float32")

# 5) Quality flag: enough impressions in the session
emp["enough_expo"] = emp["n_total"] >= min_exposures

# Save + quick peek
emp_path = OUT_PREP / "empirical_category_distribution.csv"
emp.to_csv(emp_path, index=False)
print(f"Saved empirical per-session category distribution → {emp_path}")
display(emp.head(10))


/var/folders/kx/mcwvbnt12wl9fy5xcs5jlnx00000gn/T/ipykernel_12303/3363662624.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = (expo.groupby(["user_id","session",cat_col], as_index=False)


Saved empirical per-session category distribution → /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/prepared/empirical_category_distribution.csv


,user_id,session,i_top_category_id,n_exposed,n_total,K_session,p_empirical_cat,p_empirical_cat_sm,enough_expo
0,0,1,-124,0,6,40,0.0000,0.0016,False
1,0,1,1,0,6,40,0.0000,0.0016,False
2,0,1,10,0,6,40,0.0000,0.0016,False
3,0,1,11,0,6,40,0.0000,0.0016,False
4,0,1,12,0,6,40,0.0000,0.0016,False
5,0,1,13,0,6,40,0.0000,0.0016,False
6,0,1,14,0,6,40,0.0000,0.0016,False
7,0,1,15,0,6,40,0.0000,0.0016,False
8,0,1,16,0,6,40,0.0000,0.0016,False
9,0,1,17,0,6,40,0.0000,0.0016,False


In [19]:
# --- prerequisites assumed to exist ---
# b1: DataFrame of burst-1 exposures with all 79 cols you listed
# feat_cols: the 64 feature names used to train LightGBM (exact list/order below)

import numpy as np
import pandas as pd

# ---------- 1) pick first user's first session ----------
first_pair = (b1[["user_id","session","ts"]]
              .sort_values(["user_id","session","ts"])
              .drop_duplicates(subset=["user_id","session"])
              .iloc[0])
u0 = int(first_pair["user_id"])
s0 = int(first_pair["session"])

g = (b1[(b1.user_id==u0) & (b1.session==s0)]
       .sort_values("ts", kind="mergesort")
       .reset_index(drop=True))
if g.empty:
    raise RuntimeError("No rows for first user's first session")

# session anchor time (use session start)
t_session = pd.Timestamp(g["ts"].iloc[0])

# ---------- 2) build a per-video item bank from b1 (keeps dtypes) ----------
item_cols = ["video_id","i_author_id","i_video_type","i_upload_type","i_visible_status",
             "video_width","video_height","i_music_id","i_video_tag_id","i_video_duration_s",
             "i_aspect_ratio","i_top_category_id","i_age_since_upload_days","ts"]
# keep only available columns
item_cols = [c for c in item_cols if c in b1.columns]

item_bank = (b1[item_cols]
             .sort_values("ts", kind="mergesort")
             .drop_duplicates(subset=["video_id"], keep="first")
             .reset_index(drop=True))

# estimate each video's upload time ≈ first_seen_ts - first_seen_age_days
if {"ts","i_age_since_upload_days"}.issubset(item_bank.columns):
    upload_ts_est = item_bank["ts"] - pd.to_timedelta(item_bank["i_age_since_upload_days"].astype(float), unit="D")
    item_bank = item_bank.assign(_upload_ts_est=upload_ts_est)
else:
    # fallback if age column missing: assume "today" at session time (age=0)
    item_bank = item_bank.assign(_upload_ts_est=pd.Timestamp(t_session))

# we won't keep the helper columns in final matrix
# (but keep them around here for recomputing per-session item age)
# ---------- 3) session-level features to broadcast (exactly as in b1) ----------
sess_row = g.iloc[0]  # one row from this session to copy/broadcast session/user/history fields

broadcast_cols = [
 'u_user_active_degree','u_is_lowactive_period','u_is_live_streamer','u_is_video_author',
 'u_follow_user_num','u_follow_user_num_range','u_fans_user_num','u_fans_user_num_range',
 'u_friend_user_num','u_friend_user_num_range','u_register_days','u_register_days_range',
 'u_onehot_feat0','u_onehot_feat1','u_onehot_feat2','u_onehot_feat3','u_onehot_feat4',
 'u_onehot_feat5','u_onehot_feat6','u_onehot_feat7','u_onehot_feat8','u_onehot_feat9',
 'u_onehot_feat10','u_onehot_feat11','u_onehot_feat12','u_onehot_feat13','u_onehot_feat14',
 'u_onehot_feat15','u_onehot_feat16','u_onehot_feat17','u_follow_user_num_log1p',
 'u_fans_user_num_log1p','u_friend_user_num_log1p','u_register_days_log1p',
 'ctx_hour_sin','ctx_hour_cos','ctx_dow','sess_index','prev_session_length_min',
 'inter_session_gap_hours','hist_last3_complete_rate','hist_last10_complete_rate',
 'hist_last3_wr_mean','hist_last3_wr_var','hist_ema_complete','hist_ema_wr_mean',
 'hist_prev_within_sess_wr_slope','hist_user_cat_complete_ema','hist_cat_entropy','burst_id'
]

# compute/overwrite ctx_* from actual session time to be safe
hour = t_session.hour + t_session.minute/60.0
angle = 2*np.pi*hour/24.0
ctx_hour_sin = np.sin(angle)
ctx_hour_cos = np.cos(angle)
ctx_dow = t_session.dayofweek

# build a dict of broadcast values with correct dtypes
bvals = {}
for c in broadcast_cols:
    if c in ("ctx_hour_sin","ctx_hour_cos","ctx_dow"):
        continue
    if c in g.columns:
        bvals[c] = g[c].iloc[0]
    else:
        bvals[c] = np.nan

bvals.update({"ctx_hour_sin": ctx_hour_sin, "ctx_hour_cos": ctx_hour_cos, "ctx_dow": ctx_dow})

# ---------- 4) recompute candidate-dependent features ----------
# author recency (days since this user last saw that author before t_session)
past = b1[(b1.user_id==u0) & (b1.ts < t_session)][["i_author_id","ts"]].copy()
last_seen = (past.sort_values("ts")
                  .drop_duplicates(subset=["i_author_id"], keep="last")
                  .set_index("i_author_id")["ts"])

def _author_recency_days(author_id: int) -> float:
    try:
        ts_last = last_seen.loc[author_id]
        return max((t_session - ts_last).total_seconds() / 86400.0, 0.0)
    except KeyError:
        return 365.0  # deterministic default for unseen authors

# per-candidate age at session time
def _item_age_days(up_ts: pd.Timestamp) -> float:
    return max((t_session - up_ts).total_seconds()/86400.0, 0.0)

cand = item_bank[["video_id","i_author_id","i_video_type","i_upload_type","i_visible_status",
                  "video_width","video_height","i_music_id","i_video_tag_id","i_video_duration_s",
                  "i_aspect_ratio","i_top_category_id","_upload_ts_est"]].copy()

cand["i_age_since_upload_days"] = cand["_upload_ts_est"].apply(_item_age_days).astype("float32")
cand.drop(columns=["_upload_ts_est"], inplace=True)

cand["hist_author_recency_days"]  = cand["i_author_id"].apply(_author_recency_days).astype("float32")
cand["hist_author_recency_log1p"] = np.log1p(cand["hist_author_recency_days"]).astype("float32")

# ---------- 5) assemble 2,919 × 64 matrix in the exact feat_cols order ----------
# start with per-candidate item fields
need_item = ['i_author_id','i_video_type','i_upload_type','i_visible_status','video_width',
             'video_height','i_music_id','i_video_tag_id','i_video_duration_s','i_aspect_ratio',
             'i_age_since_upload_days','i_top_category_id',
             'hist_author_recency_days','hist_author_recency_log1p']
# add placeholders for all broadcast fields
for c in broadcast_cols:
    cand[c] = bvals[c]

# ensure dtypes for categoricals match b1 (so LightGBM doesn't complain)
from pandas.api.types import CategoricalDtype
for c in ['i_author_id','i_video_type','i_upload_type','i_visible_status','i_music_id',
          'i_video_tag_id','i_top_category_id']:
    if c in b1.columns and pd.api.types.is_categorical_dtype(b1[c].dtype):
        cat_type: CategoricalDtype = b1[c].dtype
        cand[c] = cand[c].astype(cat_type)

# add session ids if you want to keep them around (not part of the 64)
cand["user_id"] = u0
cand["session"] = s0

# finally select the 64 features in *exact* training order
X_cand = cand[feat_cols].copy()

print("Candidate feature matrix for (user, session) =", (u0, s0))
print("Shape:", X_cand.shape)   # should be (2919, 64)
display(X_cand.head(5))


Candidate feature matrix for (user, session) = (0, 1)
Shape: (2919, 64)


/var/folders/kx/mcwvbnt12wl9fy5xcs5jlnx00000gn/T/ipykernel_12303/452101178.py:143: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if c in b1.columns and pd.api.types.is_categorical_dtype(b1[c].dtype):


,u_user_active_degree,u_is_lowactive_period,u_is_live_streamer,u_is_video_author,u_follow_user_num,u_follow_user_num_range,u_fans_user_num,u_fans_user_num_range,u_friend_user_num,u_friend_user_num_range,...,hist_last3_wr_mean,hist_last3_wr_var,hist_ema_complete,hist_ema_wr_mean,hist_prev_within_sess_wr_slope,hist_author_recency_days,hist_author_recency_log1p,hist_user_cat_complete_ema,hist_cat_entropy,burst_id
0,high_active,0,0,0,5.0000,"(0,10]",0.0000,0,0.0000,0,...,NaN,NaN,NaN,NaN,NaN,365.0000,5.9026,NaN,NaN,1
1,high_active,0,0,0,5.0000,"(0,10]",0.0000,0,0.0000,0,...,NaN,NaN,NaN,NaN,NaN,365.0000,5.9026,NaN,NaN,1
2,high_active,0,0,0,5.0000,"(0,10]",0.0000,0,0.0000,0,...,NaN,NaN,NaN,NaN,NaN,365.0000,5.9026,NaN,NaN,1
3,high_active,0,0,0,5.0000,"(0,10]",0.0000,0,0.0000,0,...,NaN,NaN,NaN,NaN,NaN,365.0000,5.9026,NaN,NaN,1
4,high_active,0,0,0,5.0000,"(0,10]",0.0000,0,0.0000,0,...,NaN,NaN,NaN,NaN,NaN,365.0000,5.9026,NaN,NaN,1


In [20]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

# =========================================================
# 0) One-time item bank (keeps training dtypes)
# =========================================================
_item_cols = ["video_id","i_author_id","i_video_type","i_upload_type","i_visible_status",
              "video_width","video_height","i_music_id","i_video_tag_id","i_video_duration_s",
              "i_aspect_ratio","i_top_category_id","i_age_since_upload_days","ts"]
_item_cols = [c for c in _item_cols if c in b1.columns]

ITEM_BANK = (b1[_item_cols]
             .sort_values("ts", kind="mergesort")
             .drop_duplicates(subset=["video_id"], keep="first")
             .reset_index(drop=True))

# estimate upload timestamp from first-seen age
if {"ts","i_age_since_upload_days"}.issubset(ITEM_BANK.columns):
    ITEM_BANK["_upload_ts_est"] = (
        ITEM_BANK["ts"] - pd.to_timedelta(ITEM_BANK["i_age_since_upload_days"].astype(float), unit="D")
    )
else:
    ITEM_BANK["_upload_ts_est"] = ITEM_BANK["ts"]  # harmless fallback

# which columns will be broadcast per-session
BROADCAST_COLS = [
 'u_user_active_degree','u_is_lowactive_period','u_is_live_streamer','u_is_video_author',
 'u_follow_user_num','u_follow_user_num_range','u_fans_user_num','u_fans_user_num_range',
 'u_friend_user_num','u_friend_user_num_range','u_register_days','u_register_days_range',
 'u_onehot_feat0','u_onehot_feat1','u_onehot_feat2','u_onehot_feat3','u_onehot_feat4',
 'u_onehot_feat5','u_onehot_feat6','u_onehot_feat7','u_onehot_feat8','u_onehot_feat9',
 'u_onehot_feat10','u_onehot_feat11','u_onehot_feat12','u_onehot_feat13','u_onehot_feat14',
 'u_onehot_feat15','u_onehot_feat16','u_onehot_feat17','u_follow_user_num_log1p',
 'u_fans_user_num_log1p','u_friend_user_num_log1p','u_register_days_log1p',
 'ctx_hour_sin','ctx_hour_cos','ctx_dow','sess_index','prev_session_length_min',
 'inter_session_gap_hours','hist_last3_complete_rate','hist_last10_complete_rate',
 'hist_last3_wr_mean','hist_last3_wr_var','hist_ema_complete','hist_ema_wr_mean',
 'hist_prev_within_sess_wr_slope','hist_user_cat_complete_ema','hist_cat_entropy','burst_id'
]

# item fields needed per candidate (including author recency placeholders we compute below)
ITEM_NEED = ['i_author_id','i_video_type','i_upload_type','i_visible_status','video_width',
             'video_height','i_music_id','i_video_tag_id','i_video_duration_s','i_aspect_ratio',
             'i_top_category_id']

# categories to enforce dtype equality with training
CAT_ENFORCE = ['i_author_id','i_video_type','i_upload_type','i_visible_status',
               'i_music_id','i_video_tag_id','i_top_category_id']

# =========================================================
# 1) per-session builder
# =========================================================
def build_candidate_matrix_for_session(u: int, s: int, *, return_ids: bool=False):
    """
    Returns X (2919 x 64) for (u,s). If return_ids=True, also returns ids DF with user_id/session/video_id.
    """
    g = (b1[(b1.user_id==u) & (b1.session==s)]
           .sort_values("ts", kind="mergesort")
           .reset_index(drop=True))
    if g.empty:
        return (pd.DataFrame(), pd.DataFrame()) if return_ids else pd.DataFrame()

    # anchor to session start
    t_session = pd.Timestamp(g["ts"].iloc[0])

    # ---- broadcast session/user/history ----
    hour = t_session.hour + t_session.minute/60.0
    angle = 2*np.pi*hour/24.0
    bvals = {c: (g[c].iloc[0] if c in g.columns else np.nan) for c in BROADCAST_COLS}
    bvals.update({"ctx_hour_sin": np.sin(angle), "ctx_hour_cos": np.cos(angle), "ctx_dow": t_session.dayofweek})

    # ---- author recency relative to this session ----
    past = b1[(b1.user_id==u) & (b1.ts < t_session)][["i_author_id","ts"]]
    last_seen = (past.sort_values("ts", kind="mergesort")
                      .drop_duplicates(subset=["i_author_id"], keep="last")
                      .set_index("i_author_id")["ts"])

    def _author_recency_days(aid):
        try:
            ts_last = last_seen.loc[aid]
            return max((t_session - ts_last).total_seconds()/86400.0, 0.0)
        except KeyError:
            return 365.0  # deterministic default

    # ---- assemble candidates from bank ----
    cand = ITEM_BANK[["video_id"] + ITEM_NEED + ["_upload_ts_est"]].copy()
    # per-candidate age at this session
    cand["i_age_since_upload_days"] = (
        (t_session - cand["_upload_ts_est"]).dt.total_seconds()/86400.0
    ).clip(lower=0).astype("float32")
    cand.drop(columns=["_upload_ts_est"], inplace=True)

    # author recency
    cand["hist_author_recency_days"]  = cand["i_author_id"].apply(_author_recency_days).astype("float32")
    cand["hist_author_recency_log1p"] = np.log1p(cand["hist_author_recency_days"]).astype("float32")

    # broadcast session fields
    for c, v in bvals.items():
        cand[c] = v

    # enforce categorical dtypes to match training data
    for c in CAT_ENFORCE:
        if c in b1.columns and isinstance(b1[c].dtype, CategoricalDtype):
            cand[c] = cand[c].astype(b1[c].dtype)

    # final 64 in exact order
    X = cand[feat_cols].copy()

    if return_ids:
        ids = cand[["video_id"]].copy()
        ids.insert(0, "session", s)
        ids.insert(0, "user_id", u)
        return X, ids
    return X

# =========================================================
# 2) grab Day-4, take the first 10 sessions, build giant matrix
# =========================================================
# find Day-4 (4th unique calendar day)
uniq_days = np.sort(b1["ts"].dt.normalize().unique())
if len(uniq_days) < 4:
    raise RuntimeError("Less than 4 days available in b1.")
d4 = uniq_days[3]

d4_sessions = (b1.loc[b1["ts"].dt.normalize()==d4, ["user_id","session","ts"]]
                 .sort_values(["ts"], kind="mergesort")
                 .drop_duplicates(subset=["user_id","session"])
                 .head(10)[["user_id","session"]]
                 .reset_index(drop=True))

blocks = []
id_blocks = []
for u, s in d4_sessions.itertuples(index=False):
    Xs, ids = build_candidate_matrix_for_session(int(u), int(s), return_ids=True)
    if Xs.empty:
        continue
    blocks.append(Xs)
    id_blocks.append(ids)

X_10 = pd.concat(blocks, axis=0, ignore_index=True) if blocks else pd.DataFrame(columns=feat_cols)
ids_10 = pd.concat(id_blocks, axis=0, ignore_index=True) if id_blocks else pd.DataFrame(columns=["user_id","session","video_id"])

print("Selected Day-4:", pd.to_datetime(d4).date())
print("Sessions used:", len(d4_sessions))
print("Feature matrix shape (expect 10*2919 x 64):", X_10.shape)
display(X_10.head())

# (optional) join ids for inspection only (keep features separate for model input)
preview = ids_10.join(X_10, how="left")
display(preview.head())


Selected Day-4: 2020-07-07
Sessions used: 10
Feature matrix shape (expect 10*2919 x 64): (29190, 64)


,u_user_active_degree,u_is_lowactive_period,u_is_live_streamer,u_is_video_author,u_follow_user_num,u_follow_user_num_range,u_fans_user_num,u_fans_user_num_range,u_friend_user_num,u_friend_user_num_range,...,hist_last3_wr_mean,hist_last3_wr_var,hist_ema_complete,hist_ema_wr_mean,hist_prev_within_sess_wr_slope,hist_author_recency_days,hist_author_recency_log1p,hist_user_cat_complete_ema,hist_cat_entropy,burst_id
0,full_active,0,0,0,2001.0000,500+,3.0000,"[1,10)",2.0000,"[1,5)",...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
1,full_active,0,0,0,2001.0000,500+,3.0000,"[1,10)",2.0000,"[1,5)",...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
2,full_active,0,0,0,2001.0000,500+,3.0000,"[1,10)",2.0000,"[1,5)",...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
3,full_active,0,0,0,2001.0000,500+,3.0000,"[1,10)",2.0000,"[1,5)",...,1.0996,0.2604,0.7937,1.5927,NaN,0.9928,0.6895,0.8747,2.7236,1
4,full_active,0,0,0,2001.0000,500+,3.0000,"[1,10)",2.0000,"[1,5)",...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1


,user_id,session,video_id,u_user_active_degree,u_is_lowactive_period,u_is_live_streamer,u_is_video_author,u_follow_user_num,u_follow_user_num_range,u_fans_user_num,...,hist_last3_wr_mean,hist_last3_wr_var,hist_ema_complete,hist_ema_wr_mean,hist_prev_within_sess_wr_slope,hist_author_recency_days,hist_author_recency_log1p,hist_user_cat_complete_ema,hist_cat_entropy,burst_id
0,6605,8,1907,full_active,0,0,0,2001.0000,500+,3.0000,...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
1,6605,8,3634,full_active,0,0,0,2001.0000,500+,3.0000,...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
2,6605,8,8203,full_active,0,0,0,2001.0000,500+,3.0000,...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1
3,6605,8,183,full_active,0,0,0,2001.0000,500+,3.0000,...,1.0996,0.2604,0.7937,1.5927,NaN,0.9928,0.6895,0.8747,2.7236,1
4,6605,8,139,full_active,0,0,0,2001.0000,500+,3.0000,...,1.0996,0.2604,0.7937,1.5927,NaN,365.0000,5.9026,0.8747,2.7236,1


In [22]:
import json, numpy as np, pandas as pd, lightgbm as lgb
from pandas.api.types import is_categorical_dtype

# ---- 0) Reference df used in training (for category sets & dtypes) ----
# If you still have `train` from the head training cell, use that.
# Otherwise, approximate with the subset of b1 you trained on (e.g., D1–D3).
try:
    ref_df = train[feat_cols].copy()
except NameError:
    # Fallback: use first three unique days as the training proxy
    uniq_days = np.sort(b1["ts"].dt.normalize().unique())
    train_days = set(uniq_days[:3])
    ref_df = b1.loc[b1["ts"].dt.normalize().isin(train_days), feat_cols].copy()

# categorical features as used in training
if "cat_cols" in globals():
    cat_cols_ref = [c for c in cat_cols if c in feat_cols]
else:
    # Infer: any column that was category dtype during training
    cat_cols_ref = [c for c in feat_cols if c in ref_df.columns and is_categorical_dtype(ref_df[c].dtype)]

# ---- 1) Align function: make X_pred match training dtypes & categories ----
def align_to_training_schema(X_pred: pd.DataFrame, ref: pd.DataFrame,
                             feat_cols: list[str], cat_cols: list[str]) -> pd.DataFrame:
    X = X_pred.copy()
    # keep only and order by feat_cols
    X = X[feat_cols].copy()
    # cast categorical columns to the exact categories & dtype from training
    for c in cat_cols:
        if c not in X.columns or c not in ref.columns:
            continue
        # if ref has categories, use them; otherwise just cast dtype
        if is_categorical_dtype(ref[c].dtype):
            cats = ref[c].cat.categories
            X[c] = pd.Categorical(X[c], categories=cats)  # unseen labels -> NaN
        else:
            X[c] = X[c].astype(ref[c].dtype, copy=False)
    # for non-categorical columns, ensure numeric dtype compatibility
    for c in feat_cols:
        if c in cat_cols:
            continue
        if c in ref.columns:
            try:
                X[c] = X[c].astype(ref[c].dtype, copy=False)
            except Exception:
                pass  # leave as-is if safe cast fails
    return X

# ---- 2) Load models + calibrators once ----
heads = ["y_complete","y_long","y_rewatch","y_neg"]

def _load_head(head_name: str):
    head_dir = OUTDIR / head_name
    booster = lgb.Booster(model_file=str(head_dir / f"lgb_b1_{head_name}.txt"))
    with open(head_dir / "calibrator.json") as f:
        c = json.load(f)
    return booster, float(c.get("coef", 1.0)), float(c.get("intercept", 0.0))

MODELS = {h: _load_head(h) for h in heads}

def _predict_head(head_name: str, X_feat: pd.DataFrame) -> np.ndarray:
    booster, a, b = MODELS[head_name]
    # IMPORTANT: disable LightGBM’s internal feature-name/category checks?
    # Not recommended. Instead, ensure exact schema using align_to_training_schema above.
    margin = booster.predict(X_feat, num_iteration=booster.best_iteration, raw_score=True)
    return 1.0 / (1.0 + np.exp(-(a * margin + b)))

# ---- 3) Build candidates for the first Day-4 session and ALIGN schema ----
uniq_days = np.sort(b1["ts"].dt.normalize().unique())
d4 = uniq_days[3]
first_sess = (b1.loc[b1["ts"].dt.normalize()==d4, ["user_id","session","ts"]]
                .sort_values("ts", kind="mergesort")
                .drop_duplicates(subset=["user_id","session"])
                .iloc[0])
u0, s0 = int(first_sess.user_id), int(first_sess.session)

# Your helper from earlier that returns a (2919 x 64) feature matrix for (u0,s0)
X_cand_raw, ids_cand = build_candidate_matrix_for_session(u0, s0, return_ids=True)

# Align EXACTLY to the training schema (order, dtypes, categories)
X_cand = align_to_training_schema(X_cand_raw, ref_df, feat_cols, cat_cols_ref)
assert list(X_cand.columns) == list(feat_cols), "Feature order mismatch after alignment"

# ---- 4) Predict all four heads (2919 x 4) ----
preds = {h: _predict_head(h, X_cand) for h in heads}
pred_tbl = pd.DataFrame(preds)
pred_tbl.insert(0, "video_id", ids_cand["video_id"])
print("Prediction table shape:", pred_tbl.shape)  # expect (2919, 5 with video_id)
display(pred_tbl.head())


Prediction table shape: (2919, 5)


/var/folders/kx/mcwvbnt12wl9fy5xcs5jlnx00000gn/T/ipykernel_12303/120144092.py:33: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if is_categorical_dtype(ref[c].dtype):


,video_id,y_complete,y_long,y_rewatch,y_neg
0,1907,0.8024,0.6203,0.1820,0.0552
1,3634,0.6712,0.5054,0.1327,0.0586
2,8203,0.9257,0.8184,0.4729,0.0518
3,183,0.8963,0.7513,0.3290,0.0538
4,139,0.7629,0.5759,0.1141,0.0530


In [30]:
# === Fast D4 head prediction over all eligible sessions (CategoricalDtype-safe) ===
import json, numpy as np, pandas as pd, lightgbm as lgb
from pandas.api.types import CategoricalDtype
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import warnings
warnings.filterwarnings("ignore")
# -----------------------------------------------------------------------------
# 1) Load head models + Platt calibrators once
# -----------------------------------------------------------------------------
heads = ["y_complete","y_long","y_rewatch","y_neg"]

def _load_head(head_name: str):
    head_dir = OUTDIR / head_name
    booster = lgb.Booster(model_file=str(head_dir / f"lgb_b1_{head_name}.txt"))
    with open(head_dir / "calibrator.json") as f:
        c = json.load(f)
    return booster, float(c.get("coef", 1.0)), float(c.get("intercept", 0.0))

MODELS = {h: _load_head(h) for h in heads}

def _predict_head(head_name: str, X_feat: pd.DataFrame) -> np.ndarray:
    booster, a, b = MODELS[head_name]
    margin = booster.predict(X_feat, num_iteration=booster.best_iteration, raw_score=True)
    return 1.0 / (1.0 + np.exp(-(a*margin + b)))  # Platt-calibrated prob

# -----------------------------------------------------------------------------
# 2) Reference schema for alignment (from your training split)
# -----------------------------------------------------------------------------
try:
    ref_df = train[feat_cols].copy()
except NameError:
    uniq_days = np.sort(b1["ts"].dt.normalize().unique())
    ref_df = b1.loc[b1["ts"].dt.normalize().isin(uniq_days[:3]), feat_cols].copy()

cat_cols_ref = [c for c in feat_cols
                if c in ref_df.columns and isinstance(ref_df[c].dtype, CategoricalDtype)]

def align_to_training_schema(X_pred: pd.DataFrame) -> pd.DataFrame:
    """Match training column order, categorical vocab, and numeric dtypes."""
    X = X_pred.reindex(columns=feat_cols, fill_value=np.nan).copy()
    # enforce categorical vocab
    for c in cat_cols_ref:
        X[c] = pd.Categorical(X[c], categories=ref_df[c].cat.categories)
    # cast numerics like training
    for c in feat_cols:
        if c in ref_df.columns and not isinstance(ref_df[c].dtype, CategoricalDtype):
            try:
                X[c] = X[c].astype(ref_df[c].dtype, copy=False)
            except Exception:
                pass
    return X

# -----------------------------------------------------------------------------
# 3) Cache item-static features for all 2919 videos
# -----------------------------------------------------------------------------
ITEM_STATIC_COLS = [
    "video_id","i_author_id","i_video_type","i_upload_type","i_visible_status",
    "video_width","video_height","i_music_id","i_video_tag_id",
    "i_video_duration_s","i_aspect_ratio","i_age_since_upload_days","i_top_category_id"
]
ITEM_STATIC = b1[ITEM_STATIC_COLS].drop_duplicates("video_id").reset_index(drop=True)

# -----------------------------------------------------------------------------
# 4) Fast builder for one session’s 2919×64 matrix (recompute only author-recency)
# -----------------------------------------------------------------------------
SESSION_BROADCAST_COLS = [
    'u_user_active_degree','u_is_lowactive_period','u_is_live_streamer','u_is_video_author',
    'u_follow_user_num','u_follow_user_num_range','u_fans_user_num','u_fans_user_num_range',
    'u_friend_user_num','u_friend_user_num_range','u_register_days','u_register_days_range',
    'u_onehot_feat0','u_onehot_feat1','u_onehot_feat2','u_onehot_feat3','u_onehot_feat4','u_onehot_feat5',
    'u_onehot_feat6','u_onehot_feat7','u_onehot_feat8','u_onehot_feat9','u_onehot_feat10','u_onehot_feat11',
    'u_onehot_feat12','u_onehot_feat13','u_onehot_feat14','u_onehot_feat15','u_onehot_feat16','u_onehot_feat17',
    'u_follow_user_num_log1p','u_fans_user_num_log1p','u_friend_user_num_log1p','u_register_days_log1p',
    'ctx_hour_sin','ctx_hour_cos','ctx_dow',
    'sess_index','prev_session_length_min','inter_session_gap_hours',
    'hist_last3_complete_rate','hist_last10_complete_rate','hist_last3_wr_mean','hist_last3_wr_var',
    'hist_ema_complete','hist_ema_wr_mean','hist_prev_within_sess_wr_slope',
    'hist_user_cat_complete_ema','hist_cat_entropy',
    'burst_id'
]
RANK_DEFAULT_COLS = ['sess_rank','sess_len','sess_rank_frac']  # deterministic defaults

def build_candidate_matrix_for_session_fast(u: int, s: int, return_ids: bool = False):
    g = b1[(b1.user_id == u) & (b1.session == s)]
    if g.empty:
        return (pd.DataFrame(columns=feat_cols), pd.DataFrame(columns=["video_id"])) if return_ids else pd.DataFrame(columns=feat_cols)

    row0 = g.iloc[0]
    sess_start = g["ts"].min()

    # base: all items
    C = ITEM_STATIC.copy()

    # broadcast session/user/history cols
    for c in SESSION_BROADCAST_COLS:
        C[c] = row0[c] if c in b1.columns else pd.NA

    # deterministic defaults for rank-ish features
    sess_len = int(g["sess_len"].iloc[0]) if "sess_len" in g.columns else int(g.shape[0])
    C["sess_len"] = sess_len
    C["sess_rank"] = sess_len + 1
    C["sess_rank_frac"] = 1.0

    # recompute author recency per candidate
    prior = (b1[(b1.user_id == u) & (b1["ts"] < sess_start)]
             .groupby("i_author_id", as_index=False)["ts"].max()
             .rename(columns={"ts": "last_seen_ts"}))
    C = C.merge(prior, on="i_author_id", how="left")
    rec_days = (sess_start - C["last_seen_ts"]).dt.total_seconds() / (3600*24)
    C["hist_author_recency_days"]  = rec_days.fillna(1e6).astype("float32")
    C["hist_author_recency_log1p"] = np.log1p(C["hist_author_recency_days"].clip(lower=0)).astype("float32")
    C.drop(columns=["last_seen_ts"], inplace=True)

    X = align_to_training_schema(C)
    if return_ids:
        return X, C[["video_id"]]
    return X

# -----------------------------------------------------------------------------
# 5) Per-session prediction
# -----------------------------------------------------------------------------
def predict_heads_for_session(u: int, s: int) -> pd.DataFrame:
    X, ids = build_candidate_matrix_for_session_fast(u, s, return_ids=True)
    if X.empty:
        return pd.DataFrame(columns=["user_id","session","video_id"] + heads)
    preds = {h: _predict_head(h, X) for h in heads}
    out = pd.DataFrame(preds)
    out.insert(0, "video_id", ids["video_id"].values)
    out.insert(0, "session", s)
    out.insert(0, "user_id", u)
    return out

# -----------------------------------------------------------------------------
# 6) Multi-threaded D4 runner
# -----------------------------------------------------------------------------
def predict_heads_for_dayindex_fast(day_idx: int,
                                    *, min_exposures: int = 3,
                                    max_sessions: int | None = None,
                                    max_workers: int = 6,
                                    save_dir: Path | None = None) -> pd.DataFrame:
    uniq_days = np.sort(b1["ts"].dt.normalize().unique())
    if day_idx < 0 or day_idx >= len(uniq_days):
        raise IndexError(f"day_idx out of range; have {len(uniq_days)} unique days.")
    d = uniq_days[day_idx]
    day_mask = b1["ts"].dt.normalize() == d

    sess_counts = (b1.loc[day_mask, ["user_id","session"]]
                     .value_counts().rename("n_exposures").reset_index())
    keep = sess_counts[sess_counts["n_exposures"] >= min_exposures][["user_id","session"]]
    if max_sessions is not None and len(keep) > max_sessions:
        keep = keep.sample(n=max_sessions, random_state=42).reset_index(drop=True)

    out_path = None
    if save_dir is not None:
        save_dir.mkdir(parents=True, exist_ok=True)
        out_path = save_dir / f"head_probs_day{day_idx+1}.csv"
        if out_path.exists(): out_path.unlink()

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = [ex.submit(predict_heads_for_session, int(u), int(s))
                for u, s in keep.itertuples(index=False)]
        for i, fut in enumerate(as_completed(futs), 1):
            df_i = fut.result()
            if df_i is not None and not df_i.empty:
                if out_path is not None:
                    df_i.to_csv(out_path, mode="a", index=False, header=not out_path.exists())
                results.append(df_i)
            if i % 500 == 0:
                print(f"[Day {int(day_idx)+1}] finished {i}/{len(keep)} sessions …")

    return (pd.concat(results, ignore_index=True)
            if results else pd.DataFrame(columns=["user_id","session","video_id"]+heads))

# -----------------------------------------------------------------------------
# 7) Run for D4 and preview
# -----------------------------------------------------------------------------
d4_idx = 3  # 4th unique day
save_dir = OUTDIR / "policy" / "heads_by_day_fast"

pred_d4 = predict_heads_for_dayindex_fast(d4_idx, min_exposures=3, max_sessions=None,
                                          max_workers=6, save_dir=save_dir)
print("D4 predictions shape:", pred_d4.shape)
display(pred_d4.head())


[Day 4] finished 50/15455 sessions …
[Day 4] finished 100/15455 sessions …
[Day 4] finished 150/15455 sessions …
[Day 4] finished 200/15455 sessions …
[Day 4] finished 250/15455 sessions …
[Day 4] finished 300/15455 sessions …
[Day 4] finished 350/15455 sessions …
[Day 4] finished 400/15455 sessions …
[Day 4] finished 450/15455 sessions …
[Day 4] finished 500/15455 sessions …
[Day 4] finished 550/15455 sessions …
[Day 4] finished 600/15455 sessions …
[Day 4] finished 650/15455 sessions …
[Day 4] finished 700/15455 sessions …
[Day 4] finished 750/15455 sessions …
[Day 4] finished 800/15455 sessions …
[Day 4] finished 850/15455 sessions …
[Day 4] finished 900/15455 sessions …
[Day 4] finished 950/15455 sessions …
[Day 4] finished 1000/15455 sessions …
[Day 4] finished 1050/15455 sessions …
[Day 4] finished 1100/15455 sessions …
[Day 4] finished 1150/15455 sessions …
[Day 4] finished 1200/15455 sessions …
[Day 4] finished 1250/15455 sessions …
[Day 4] finished 1300/15455 sessions …
[Day 4

,user_id,session,video_id,y_complete,y_long,y_rewatch,y_neg
0,1092,10,3649,0.2406,0.1994,0.0744,0.4881
1,1092,10,9598,0.2189,0.2007,0.0351,0.5077
2,1092,10,5262,0.2903,0.1933,0.1255,0.5696
3,1092,10,1963,0.2686,0.2281,0.0994,0.5218
4,1092,10,8234,0.2020,0.1632,0.0620,0.5027


In [32]:
from pathlib import Path
import numpy as np
import pandas as pd

# --- 1) Basic shape & sanity checks ---
print("pred_d4 shape:", pred_d4.shape)

# unique sessions & videos
n_sessions = pred_d4[["user_id","session"]].drop_duplicates().shape[0]
n_videos   = pred_d4["video_id"].nunique()
print(f"Unique sessions: {n_sessions:,}")
print(f"Unique videos:   {n_videos:,}")

# per-session row counts (should all be N_videos)
per_sess_counts = pred_d4.groupby(["user_id","session"]).size()
print("Per-session item count  min/max/mean:",
      per_sess_counts.min(), per_sess_counts.max(), int(per_sess_counts.mean()))

# expected total rows check
expected_rows = n_sessions * n_videos
print(f"Expected rows = {expected_rows:,}")
assert len(pred_d4) == expected_rows, "Row count mismatch! (some sessions may be missing videos)"

# probability range checks
for c in ["y_complete","y_long","y_rewatch","y_neg"]:
    m, M = pred_d4[c].min(), pred_d4[c].max()
    print(f"{c}: min={m:.6f}, max={M:.6f}")
    assert np.isfinite(pred_d4[c]).all() and (pred_d4[c].between(0,1).all()), f"{c} out of [0,1]!"

# memory footprint
mem_gb = pred_d4.memory_usage(deep=True).sum() / 1e9
print(f"Approx. memory (GB): {mem_gb:0.2f}")

# --- 2) Save (Parquet, compressed) ---
save_dir = OUTDIR / "policy" / "heads_by_day_fast"
save_dir.mkdir(parents=True, exist_ok=True)

# downcast to float32 to cut size ~50%
for c in ["y_complete","y_long","y_rewatch","y_neg"]:
    pred_d4[c] = pred_d4[c].astype("float32")

out_path = save_dir / "heads_D4.parquet"
pred_d4.to_parquet(out_path, index=False)  # snappy compression via pyarrow
print("Saved head predictions to:", out_path)

# manifest: rows per session (handy for fast reads later)
manifest = (pred_d4.groupby(["user_id","session"]).size()
            .rename("n_rows").reset_index())
manifest_path = save_dir / "heads_D4_manifest.csv"
manifest.to_csv(manifest_path, index=False)
print("Saved manifest to:", manifest_path)


pred_d4 shape: (45113145, 7)
Unique sessions: 15,455
Unique videos:   2,919
Per-session item count  min/max/mean: 2919 2919 2919
Expected rows = 45,113,145
y_complete: min=0.002257, max=0.999492
y_long: min=0.000544, max=0.998993
y_rewatch: min=0.000137, max=0.847327
y_neg: min=0.000068, max=0.984444
Approx. memory (GB): 2.57
Saved head predictions to: /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/models/burst1/policy/heads_by_day_fast/heads_D4.parquet
Saved manifest to: /Users/haozhangao/Desktop/RecSys Research/KuaiRec 2.0/data/models/burst1/policy/heads_by_day_fast/heads_D4_manifest.csv


In [34]:
import numpy as np
import pandas as pd

# --- Inputs assumed available ---
# pred_d4: columns ["user_id","session","video_id","y_complete","y_long","y_rewatch","y_neg"]
# b1:      has ["video_id","i_top_category_id"]

# 0) Build a stable video_id -> category map (one row per video)
vid2cat = (
    b1[["video_id","i_top_category_id"]]
      .drop_duplicates("video_id")
      .set_index("video_id")["i_top_category_id"]
)

# 1) Helpers
def softmax1d(u: np.ndarray, tau: float = 1.0) -> np.ndarray:
    """Numerically-stable softmax over a 1D array u / tau."""
    z = u / float(tau)
    m = np.max(z)
    p = np.exp(z - m)
    p /= p.sum()
    return p

def session_item_and_category_probs(
    df_heads_session: pd.DataFrame,
    vid2cat: pd.Series,
    w: np.ndarray,
    tau: float = 1.0
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Given head predictions for ONE session (2919 rows) and weights w (len=4),
    return:
      - item_probs:  [video_id, p_item]
      - cat_probs:   [i_top_category_id, p_cat]  (sum over items in each category)
    """
    # compute utilities U_j = w^T * head_probs_j
    H = df_heads_session[["y_complete","y_long","y_rewatch","y_neg"]].to_numpy(dtype="float64")
    U = H @ w

    # item softmax
    p_item = softmax1d(U, tau=tau)

    item_probs = df_heads_session[["video_id"]].copy()
    item_probs["p_item"] = p_item.astype("float64")

    # attach category per video and aggregate
    item_probs["i_top_category_id"] = item_probs["video_id"].map(vid2cat).values
    cat_probs = (item_probs.groupby("i_top_category_id", as_index=False)["p_item"]
                           .sum()
                           .rename(columns={"p_item":"p_cat"}))
    # normalize defensively (should already sum to 1)
    s = float(cat_probs["p_cat"].sum())
    if s > 0:
        cat_probs["p_cat"] = cat_probs["p_cat"] / s

    return item_probs, cat_probs

# 2) Test on the FIRST D4 session
# pick the first (user_id, session) pair in pred_d4
first_pair = pred_d4[["user_id","session"]].iloc[0].to_list()
u0, s0 = int(first_pair[0]), int(first_pair[1])

df_one = pred_d4[(pred_d4.user_id==u0) & (pred_d4.session==s0)].copy()
print(f"Testing on session (user_id={u0}, session={s0})  rows={len(df_one)}")

# example weights (you can change later)
w_test = np.array([0.05, 0.03, 0.02, -0.05], dtype="float64")
tau = 1.0

item_probs, cat_probs = session_item_and_category_probs(df_one, vid2cat, w_test, tau=tau)

# 3) Quick sanity checks / preview
print("Sum of item probs:", item_probs["p_item"].sum())
print("Sum of category probs:", cat_probs["p_cat"].sum())

print("\nTop-5 items by p_item:")
display(item_probs.sort_values("p_item", ascending=False).head(5))

print("\nTop-10 categories by p_cat:")
display(cat_probs.sort_values("p_cat", ascending=False).head(30))


Testing on session (user_id=1092, session=10)  rows=2919
Sum of item probs: 1.0
Sum of category probs: 1.0

Top-5 items by p_item:


,video_id,p_item,i_top_category_id
1045,6705,0.0004,8
1041,8143,0.0004,8
1236,9503,0.0004,8
1273,1894,0.0004,28
970,1904,0.0004,11



Top-10 categories by p_cat:


,i_top_category_id,p_cat
21,28,0.1042
34,5,0.0921
27,34,0.0848
37,8,0.0654
11,19,0.0536
3,11,0.0531
4,12,0.0444
38,9,0.0426
18,25,0.0387
1,1,0.0367


In [52]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Assumes available:
# - packs_d4: list of dicts:
#     {"user_id": int,
#      "session": int,
#      "H":  (n_items, 4)  # head probs (in order [complete,long,rewatch,neg])
#      "inv": (n_items,)   # item -> category index
#      "cats": (K,)        # category ids for this session's candidate set
#      "q":   (K,) }       # empirical category PMF, sums to 1 (smoothed)
#
# - emp: DataFrame with ["user_id","session","n_total"] (from your empirical build),
#        where n_total is the number of exposures that session had
#
# If you don’t already have the exposure counts per (u,s):
e_by_sess = (emp[["user_id","session","n_total"]]
             .drop_duplicates(["user_id","session"])
             .set_index(["user_id","session"])["n_total"]
             .to_dict())

heads = ["y_complete","y_long","y_rewatch","y_neg"]  # ordering used in H

def theta_to_w(theta: np.ndarray) -> np.ndarray:
    """
    Map unconstrained theta -> signed weights with desired semantics:
      w_c, w_l, w_r >= 0; w_neg <= 0
    """
    # theta shape (4,)
    return np.array([
        np.exp(theta[0]),   # w_complete >= 0
        np.exp(theta[1]),   # w_long     >= 0
        np.exp(theta[2]),   # w_rewatch  >= 0
        -np.exp(theta[3])   # w_neg      <= 0
    ], dtype="float64")

def topk_category_ce_for_session(P, w: np.ndarray) -> float:
    """
    One session:
      - score items: U = H @ w
      - take top-K where K = exposure count for this (user, session)
      - aggregate category distribution over those top-K (uniform 1/K each)
      - return CE(q || pi_topK) = -sum_k q_k log pi_k
    """
    u, s = P["user_id"], P["session"]
    K_s = int(e_by_sess.get((u, s), 0))
    if K_s <= 0:
        return 0.0  # session not in exposure table (shouldn't happen if packs built from emp)

    H   = P["H"]        # (n_items, 4)
    inv = P["inv"]      # (n_items,)
    q   = P["q"]        # (K_cat,)

    U = H @ w                      # (n_items,)
    # top-K indices (stable ties)
    K_take = min(K_s, U.shape[0])
    idx = np.argpartition(U, -K_take)[-K_take:]   # O(n) selection
    # If you prefer exactly sorted top-K (costlier):
    idx = idx[np.argsort(U[idx])[::-1]]

    # Aggregate category mass among the top-K (uniform over the slate)
    K_cat = q.shape[0]
    pi = np.zeros(K_cat, dtype="float64")
    for j in idx:
        pi[inv[j]] += 1.0
    pi /= K_take

    # Cross-entropy: CE(q || pi) = - sum q log pi
    pi_safe = np.clip(pi, 1e-12, 1.0)
    ce = -float(np.sum(q * np.log(pi_safe)))
    return ce

def topk_ce_loss_theta(theta: np.ndarray, packs) -> float:
    """
    Objective over all sessions (average CE) using Top-K slate matching.
    theta -> w (signed), then sum per-session CE.
    """
    w = theta_to_w(theta)
    total = 0.0
    count = 0
    for P in packs:
        # skip if we don't have an exposure count
        if (P["user_id"], P["session"]) not in e_by_sess:
            continue
        total += topk_category_ce_for_session(P, w)
        count += 1
    if count == 0:
        return np.inf
    return total / count

# -----------------------
# Quick test / fit on D4
# -----------------------
# init theta so that w starts near [0.05, 0.03, 0.02, -0.05]
theta0 = np.log(np.array([0.05, 0.03, 0.02, 0.05], dtype="float64"))  # neg uses +0.05 then flipped by theta_to_w

obj = lambda th: topk_ce_loss_theta(th, packs_d4)

# Use gradient-free optimizer (Nelder-Mead). You can also try Powell.
res = minimize(obj, theta0, method="Nelder-Mead",
               options=dict(maxiter=300, xatol=1e-4, fatol=1e-6, disp=True))

w_hat = theta_to_w(res.x)
print("Status:", res.message, "| success:", res.success)
print("Learned w:", np.round(w_hat, 6))
print("Final (avg) CE:", res.fun)

# # Sanity check: evaluate at a few hand-picked weights
# for w_try in [np.array([0.0,0.0,0.0,0.0]),
#               np.array([0.05,0.03,0.02,-0.05]),
#               np.array([0.2,0.1,0.05,-0.15])]:
#     # map back to theta just for printing a comparable value
#     th_try = np.log(np.array([max(w_try[0],1e-6),
#                               max(w_try[1],1e-6),
#                               max(w_try[2],1e-6),
#                               max(-w_try[3],1e-6)]))
#     print("w:", np.round(w_try, 4), " -> CE:", obj(th_try))


Optimization terminated successfully.
         Current function value: 16.619739
         Iterations: 59
         Function evaluations: 135
Status: Optimization terminated successfully. | success: True
Learned w: [ 0.032791  0.883349  0.007782 -0.018293]
Final (avg) CE: 16.619739359599635
w: [0. 0. 0. 0.]  -> CE: 17.384395221707404
w: [ 0.05  0.03  0.02 -0.05]  -> CE: 17.40793936081155
w: [ 0.2   0.1   0.05 -0.15]  -> CE: 17.358627996747558


In [53]:
# ================================================================
# Simulate category PMFs on D5–D9 with learned weights
# and save per-session residuals:  residual = q_empirical - p_policy
# ================================================================
import json, numpy as np, pandas as pd
from pathlib import Path

# ---- config / paths ----
cat_col   = "i_top_category_id"
heads     = ["y_complete","y_long","y_rewatch","y_neg"]
tau       = 1.0
min_exposures = 3

out_root  = OUTDIR / "policy"
pred_dir  = out_root / "heads_by_day_fast"      # where head_probs_day*.csv were saved
res_dir   = out_root / "residuals"
res_dir.mkdir(parents=True, exist_ok=True)
res_path  = res_dir / "D5D9_session_category_residuals.csv"

# ---- load weights (or hardcode) ----
pol_path = out_root / "topk_ce" / "policy_weights.json"
if pol_path.exists():
    with open(pol_path) as f:
        pol = json.load(f)
    w_hat = np.array(pol["w"], dtype="float64")
else:
    # fallback: paste your learned weights here
    w_hat = np.array([0.032791, 0.883349, 0.007782, -0.018293], dtype="float64")

print("Using weights:", np.round(w_hat, 6))

# ---- helper: load-or-build predictions for a day index (1-based label only used for file name) ----
def ensure_predictions_for_dayindex(day_idx: int) -> pd.DataFrame:
    """Return long DF: user_id, session, video_id, y_complete, y_long, y_rewatch, y_neg."""
    pred_path = pred_dir / f"head_probs_day{day_idx+1}.csv"
    if pred_path.exists():
        return pd.read_csv(pred_path)
    # If not found, fall back to computing once (uses your fast builder if it's in this notebook)
    print(f"[day{day_idx+1}] predictions not found on disk; computing …")
    return predict_heads_for_dayindex_fast(day_idx, min_exposures=min_exposures, save_dir=pred_dir)

# ---- empirical q (keep only robust sessions) ----
emp_q = (
    emp.loc[emp["n_total"] >= min_exposures, ["user_id","session",cat_col,"p_empirical_cat_sm"]]
       .rename(columns={"p_empirical_cat_sm":"q"})
       .copy()
)

# ---- item meta (video -> category) ----
item_meta = (
    b1[["video_id", cat_col]]
      .drop_duplicates("video_id")
      .reset_index(drop=True)
)

# ---- exposures per session (K_s for Top-K) ----
# We’ll grab per-day shortly to avoid pulling the whole table repeatedly.
def exposures_for_day(day_ts_norm) -> pd.DataFrame:
    return (b1.loc[b1["ts"].dt.normalize()==day_ts_norm, ["user_id","session","video_id"]]
              .drop_duplicates()
              .groupby(["user_id","session"], as_index=False)
              .size()
              .rename(columns={"size":"n_shown"}))

# ---- policy simulation on one day (Top-K slate) ----
def simulate_day_topk(day_idx: int) -> pd.DataFrame:
    """
    Returns long DF:
      user_id, session, i_top_category_id, q_emp, p_policy, residual
    for all D_s sessions on this day with >= min_exposures.
    """
    # day index -> actual calendar day
    uniq_days = np.sort(b1["ts"].dt.normalize().unique())
    d = uniq_days[day_idx]

    # predictions and exposures for the day
    pred = ensure_predictions_for_dayindex(day_idx)
    if pred.empty:
        return pd.DataFrame(columns=["user_id","session",cat_col,"q_emp","p_policy","residual","day_idx","day"])
    ks   = exposures_for_day(d)  # K_s per session
    # join categories to predictions once
    pred = pred.merge(item_meta, on="video_id", how="left")

    # restrict to sessions present in emp_q
    key_cols = ["user_id","session"]
    have_q = emp_q.merge(ks[key_cols], on=key_cols, how="inner")[key_cols].drop_duplicates()
    pred   = pred.merge(have_q, on=key_cols, how="inner")
    ks     = ks.merge(have_q, on=key_cols, how="inner")

    # Build a quick dict (u,s) -> K_s
    K_map = {(int(r.user_id), int(r.session)): int(r.n_shown) for _, r in ks.iterrows()}

    # Prepare output rows
    out_rows = []

    # Group by session
    for (u,s), g in pred.groupby(key_cols, sort=False):
        K_s = K_map.get((int(u), int(s)), None)
        if K_s is None or K_s <= 0:
            continue

        # utilities for all candidates in this session
        H = g[heads].to_numpy(dtype="float64")  # (N,4)
        U = H @ w_hat

        # pick Top-K_s indices
        if K_s < len(U):
            top_idx = np.argpartition(-U, K_s-1)[:K_s]
        else:
            top_idx = np.arange(len(U), dtype=int)

        top = g.iloc[top_idx][[cat_col]].copy()
        # aggregate to category shares within the slate
        p_cat = (top.value_counts(cat_col, normalize=True)
                     .rename("p_policy")
                     .reset_index())

        # empirical q for this session (aligned on categories)
        q_s = emp_q[(emp_q.user_id==u) & (emp_q.session==s)][[cat_col,"q"]].copy()
        q_s = q_s.rename(columns={"q":"q_emp"})

        # full outer join over categories seen in either pmf
        both = q_s.merge(p_cat, on=cat_col, how="outer")
        both["q_emp"]   = both["q_emp"].fillna(0.0)
        both["p_policy"]= both["p_policy"].fillna(0.0)
        both["residual"]= both["q_emp"] - both["p_policy"]
        both.insert(0, "session", int(s))
        both.insert(0, "user_id", int(u))
        both["day_idx"] = int(day_idx)
        both["day"]     = pd.to_datetime(d).date()

        out_rows.append(both)

    return pd.concat(out_rows, ignore_index=True) if out_rows else pd.DataFrame(
        columns=["user_id","session",cat_col,"q_emp","p_policy","residual","day_idx","day"]
    )

# ---- run for D5–D9 and stream to disk ----
if res_path.exists():
    res_path.unlink()

uniq_days = np.sort(b1["ts"].dt.normalize().unique())
# D4 index was 3 -> D5..D9 are 4..8 (guard if fewer days exist)
start_idx = 4
end_idx   = min(len(uniq_days)-1, 8)

total_rows = 0
for di in range(start_idx, end_idx+1):
    print(f"[simulate] Day index {di} ({pd.to_datetime(uniq_days[di]).date()}) …")
    resid_di = simulate_day_topk(di)
    if resid_di.empty:
        print("  nothing to write.")
        continue
    # append mode
    resid_di.to_csv(res_path, mode="a", index=False, header=not res_path.exists())
    total_rows += len(resid_di)
    print(f"  wrote {len(resid_di):,} rows (cumulative {total_rows:,}).")

print(f"\nSaved residuals → {res_path}")
# Quick peek
resid_sample = pd.read_csv(res_path, nrows=10)
display(resid_sample)

# Optional: quick summaries
resid_full = pd.read_csv(res_path, usecols=["residual"])
print("\nResidual summary (q_emp - p_policy):")
print(resid_full["residual"].describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95]).to_string())


Using weights: [ 0.032791  0.883349  0.007782 -0.018293]
[simulate] Day index 4 (2020-07-08) …
[day5] predictions not found on disk; computing …
[Day 5] finished 50/15770 sessions …
[Day 5] finished 100/15770 sessions …
[Day 5] finished 150/15770 sessions …
[Day 5] finished 200/15770 sessions …
[Day 5] finished 250/15770 sessions …
[Day 5] finished 300/15770 sessions …
[Day 5] finished 350/15770 sessions …
[Day 5] finished 400/15770 sessions …
[Day 5] finished 450/15770 sessions …
[Day 5] finished 500/15770 sessions …
[Day 5] finished 550/15770 sessions …
[Day 5] finished 600/15770 sessions …
[Day 5] finished 650/15770 sessions …
[Day 5] finished 700/15770 sessions …
[Day 5] finished 750/15770 sessions …
[Day 5] finished 800/15770 sessions …
[Day 5] finished 850/15770 sessions …
[Day 5] finished 900/15770 sessions …
[Day 5] finished 950/15770 sessions …
[Day 5] finished 1000/15770 sessions …
[Day 5] finished 1050/15770 sessions …
[Day 5] finished 1100/15770 sessions …
[Day 5] finished 

,user_id,session,i_top_category_id,q_emp,p_policy,residual,day_idx,day
0,2467,1,-124,0.0000,0.0000,0.0000,4,2020-07-08
1,2467,1,1,0.0329,0.0247,0.0082,4,2020-07-08
2,2467,1,10,0.0247,0.0000,0.0247,4,2020-07-08
3,2467,1,11,0.0535,0.0617,-0.0083,4,2020-07-08
4,2467,1,12,0.0247,0.0206,0.0041,4,2020-07-08
5,2467,1,13,0.0041,0.0370,-0.0329,4,2020-07-08
6,2467,1,14,0.0041,0.0082,-0.0041,4,2020-07-08
7,2467,1,15,0.0247,0.0082,0.0165,4,2020-07-08
8,2467,1,16,0.0041,0.0288,-0.0247,4,2020-07-08
9,2467,1,17,0.0370,0.0123,0.0247,4,2020-07-08



Residual summary (q_emp - p_policy):
count   2973200.0000
mean          0.0000
std           0.0903
min          -1.0000
10%          -0.0420
25%           0.0002
50%           0.0010
75%           0.0023
90%           0.0495
95%           0.1074
max           0.9992


In [58]:
import pandas as pd
from pathlib import Path

# Path to your saved residuals
res_path = OUTDIR / "policy" / "residuals" / "D5D9_session_category_residuals.csv"

# Load and filter out the unrecorded category
resid = pd.read_csv(res_path)
resid = resid[resid["i_top_category_id"] != -124].copy()

# --- 1) POOLED mean residual across ALL days, by category
pooled_cat_mean = (
    resid.groupby("i_top_category_id", as_index=False)["residual"]
         .mean()
         .rename(columns={"residual": "mean_residual_pooled"})
         .sort_values("mean_residual_pooled")
)
print("Pooled (D5–D9) mean residual by category (exclude -124):")
display(pooled_cat_mean.head(10))
display(pooled_cat_mean.tail(10))

# Optional: overall pooled mean across all categories and sessions (single scalar)
overall_pooled_mean = resid["residual"].mean()
print("Overall pooled mean residual (exclude -124):", overall_pooled_mean)

# --- 2) PER-DAY mean residual by category
# day column is already in the CSV; if you prefer day_idx, swap "day" with "day_idx" below
perday_cat_mean = (
    resid.groupby(["day", "i_top_category_id"], as_index=False)["residual"]
         .mean()
         .rename(columns={"residual": "mean_residual"})
)

print("\nPer-day mean residual by category (exclude -124):")
display(perday_cat_mean.head(10))

# (Nice view) Pivot to category x day matrix
perday_pivot = (
    perday_cat_mean.pivot(index="i_top_category_id", columns="day", values="mean_residual")
                   .sort_index()
)
print("\nPer-day mean residual pivot (rows=category, cols=calendar day):")
display(perday_pivot)

# --- 3) PER-DAY overall mean residual across all categories (one row per day)
perday_overall_mean = (
    resid.groupby("day", as_index=False)["residual"].mean()
         .rename(columns={"residual": "mean_residual_overall"})
)
print("\nPer-day overall mean residual (pooled across categories, exclude -124):")
display(perday_overall_mean)


Pooled (D5–D9) mean residual by category (exclude -124):


,i_top_category_id,mean_residual_pooled
37,8,-0.3090
34,5,-0.0455
27,34,-0.0223
4,12,-0.0212
10,18,-0.0159
31,38,-0.0074
33,4,-0.0050
13,20,-0.0035
23,3,-0.0019
11,19,-0.0014


,i_top_category_id,mean_residual_pooled
22,29,0.0073
12,2,0.0123
7,15,0.0134
3,11,0.0148
24,31,0.0161
2,10,0.0221
25,32,0.0263
35,6,0.0468
38,9,0.0469
21,28,0.1672


Overall pooled mean residual (exclude -124): 6.587151576343353e-12

Per-day mean residual by category (exclude -124):


,day,i_top_category_id,mean_residual
0,2020-07-08,-124,0.0018
1,2020-07-08,1,0.0078
2,2020-07-08,10,0.0261
3,2020-07-08,11,0.0071
4,2020-07-08,12,-0.0122
5,2020-07-08,13,0.0050
6,2020-07-08,14,0.0031
7,2020-07-08,15,0.0055
8,2020-07-08,16,0.0094
9,2020-07-08,17,0.0054



Per-day mean residual pivot (rows=category, cols=calendar day):


day,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12
i_top_category_id,,,,,
-124,0.0018,0.0015,0.0016,0.0018,0.0018
1,0.0078,0.0023,0.0006,0.0004,-0.0040
10,0.0261,0.0190,0.0265,0.0199,0.0178
11,0.0071,0.0047,0.0059,0.0248,0.0392
12,-0.0122,-0.0149,-0.0272,-0.0300,-0.0221
13,0.0050,0.0014,0.0023,0.0118,0.0078
14,0.0031,0.0026,0.0050,0.0064,0.0050
15,0.0055,0.0055,0.0173,0.0249,0.0143
16,0.0094,0.0073,0.0080,0.0055,0.0052



Per-day overall mean residual (pooled across categories, exclude -124):


,day,mean_residual_overall
0,2020-07-08,0.0000
1,2020-07-09,0.0000
2,2020-07-10,0.0000
3,2020-07-11,0.0000
4,2020-07-12,0.0000


In [61]:
# =========================
# Plan A: Diagnostics code
# =========================
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# -------------------------
# Load & basic filtering
# -------------------------
resid = pd.read_csv(res_path)

# Optional: drop "unknown" category if you want
resid = resid[resid["i_top_category_id"] != -124].copy()

# Sanity cleanups
resid["user_id"] = resid["user_id"].astype(int)
resid["session"] = resid["session"].astype(int)
resid["p_policy"] = resid["p_policy"].fillna(0.0)
resid["q_emp"]    = resid["q_emp"].fillna(0.0)
resid["residual"] = resid["q_emp"] - resid["p_policy"]  # defensively recompute

# ===========================================================
# 1) TOST (Two One-Sided Tests) : mean(residual) ≈ 0 ± delta
#    Clustered SEs by user
# ===========================================================
def tost_mean_zero(resid_df: pd.DataFrame, delta: float = 0.005, label: str = "All"):
    """
    Tests H0: |mu| >= delta  vs  H1: |mu| < delta  (equivalence)
    Using OLS with intercept only and user-clustered SEs.
    Prints the result and returns a dict.
    """
    y = resid_df["residual"].to_numpy()
    uid = resid_df["user_id"].to_numpy()
    X = np.ones((len(y), 1))  # intercept only
    fit = sm.OLS(y, X).fit(cov_type="cluster", cov_kwds={"groups": uid})

    mu = float(fit.params[0])
    se = float(fit.bse[0])
    df = fit.df_resid

    # TOST: need BOTH one-sided tests to be significant to claim equivalence
    t1 = (mu - (-delta)) / se   # H0: mu <= -delta  (test mu > -delta)
    t2 = (delta - mu) / se      # H0: mu >= +delta (test mu < +delta)
    p1 = 1 - t.cdf(t1, df)      # upper tail
    p2 = 1 - t.cdf(t2, df)

    p_tost = max(p1, p2)        # conservative p-value

    print(f"\n=== TOST mean≈0 (±{delta:.3f}) — {label} ===")
    print(f"mu={mu:.6f}  se={se:.6f}  df={df:.0f}")
    print(f"TOST p-value={p_tost:.3g}   "
          f"[one-sided p's: p(mu>-δ)={p1:.3g}, p(mu<+δ)={p2:.3g}]")
    print("Interpretation: p<0.05 → mean residual is within ±δ (practically zero).")

    return {"label": label, "mu": mu, "se": se, "p_tost": p_tost, "p1": p1, "p2": p2}

# Run once on pooled and also per day (D5–D9)
_ = tost_mean_zero(resid, delta=0.005, label="Pooled D5–D9")
for d in sorted(resid["day_idx"].unique()):
    tost_mean_zero(resid[resid["day_idx"] == d], delta=0.005, label=f"Day idx {d}")


# ===========================================================
# 3) Serial structure test:
#    Lag-1 autocorrelation of session-level residual sums per user
# ===========================================================
def serial_structure(resid_df: pd.DataFrame, label: str = "All"):
    """
    Collapse to (user, session) total residual, sort by (user, day_idx, session),
    compute lag-1 autocorrelation per user. Summarize the distribution.
    """
    # collapse residual vector within a session to a scalar (sum preserves sign)
    sess = (resid_df.groupby(["user_id","day_idx","session"], as_index=False)["residual"]
                  .sum()
                  .sort_values(["user_id","day_idx","session"]))

    def ac1(x):
        if len(x) < 2: return np.nan
        return pd.Series(x).autocorr(lag=1)

    ac_by_user = (sess.groupby("user_id")["residual"].apply(ac1).dropna())
    if ac_by_user.empty:
        print(f"\n=== Serial structure — {label} ===\nNot enough per-user time series.")
        return ac_by_user

    print(f"\n=== Serial structure — {label} ===")
    print(f"users={ac_by_user.size} | "
          f"median AC1={ac_by_user.median():.3f} | "
          f"p10={ac_by_user.quantile(0.10):.3f} | "
          f"p90={ac_by_user.quantile(0.90):.3f}")

    # Test mean(ac1) == 0 with a plain t-test (quick heuristic)
    m = ac_by_user.mean(); s = ac_by_user.std(ddof=1); n = ac_by_user.size
    tstat = m / (s / np.sqrt(n)) if s > 0 else np.nan
    pval  = 2 * (1 - t.cdf(abs(tstat), n-1)) if s > 0 else np.nan
    print(f"Mean(AC1)={m:.3f}  t={tstat:.3f}  p={pval:.3g}")
    print("Interpretation: small median and |mean| with non-sig p → little serial dependence.")
    return ac_by_user

_ = serial_structure(resid, label="Pooled D5–D9")
for d in sorted(resid["day_idx"].unique()):
    serial_structure(resid[resid["day_idx"] == d], label=f"Day idx {d}")



=== TOST mean≈0 (±0.005) — Pooled D5–D9 ===
mu=-0.000000  se=0.000000  df=2973199
TOST p-value=0   [one-sided p's: p(mu>-δ)=0, p(mu<+δ)=0]
Interpretation: p<0.05 → mean residual is within ±δ (practically zero).

=== TOST mean≈0 (±0.005) — Day idx 4 ===
mu=-0.000000  se=0.000000  df=630799
TOST p-value=0   [one-sided p's: p(mu>-δ)=0, p(mu<+δ)=0]
Interpretation: p<0.05 → mean residual is within ±δ (practically zero).

=== TOST mean≈0 (±0.005) — Day idx 5 ===
mu=-0.000000  se=0.000000  df=642479
TOST p-value=0   [one-sided p's: p(mu>-δ)=0, p(mu<+δ)=0]
Interpretation: p<0.05 → mean residual is within ±δ (practically zero).

=== TOST mean≈0 (±0.005) — Day idx 6 ===
mu=-0.000000  se=0.000000  df=629239
TOST p-value=0   [one-sided p's: p(mu>-δ)=0, p(mu<+δ)=0]
Interpretation: p<0.05 → mean residual is within ±δ (practically zero).

=== TOST mean≈0 (±0.005) — Day idx 7 ===
mu=-0.000000  se=0.000000  df=626879
TOST p-value=0   [one-sided p's: p(mu>-δ)=0, p(mu<+δ)=0]
Interpretation: p<0.05 → mea